In [ ]:
from google.colab import drive
ROOT = "/content/gdrive"
drive.mount(ROOT, force_remount=True)

Mounted at /content/drive


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import os
basePath = '/content/gdrive/My Drive/Colab_Notebooks/my_project1/deep_learning/S15_A/'
os.chdir(basePath)

In [ ]:
import numpy as np
import cv2
import io
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
import zipfile
import matplotlib.pyplot as plt
import torchvision
from PIL import Image
import glob
import torch
from tqdm import tqdm

class ZipDataset(Dataset):
    def __init__(self, zip_path,  transform=None):
        self.transform = transform
        f = open(zip_path, 'rb')
        self.zip_content = f.read()
        f.close()
        self.zip_handle = zipfile.ZipFile(io.BytesIO(self.zip_content), 'r')
        self.name_list = self.zip_handle.namelist()

    def __getitem__(self, key):
        buf = self.zip_handle.read(self.name_list[key])
        dataEnc = io.BytesIO(buf)
        images= np.array(Image.open(dataEnc ), dtype=float)/255.
        if self.transform:
          images = self.transform(images)
        return images

    def __len__(self):
        return len(self.name_list)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_list, transform=None):
        self.transform = transform
        self.image_list = image_list

    def __getitem__(self, key):
        images = np.array(Image.open(self.image_list[key]))/255.
        if self.transform:
          images = self.transform(images)
        return images

    def __len__(self):
        return len(self.image_list)

In [ ]:
from torchvision import transforms
def run_stastics_with_zip_files(zip_list, batch_size=256, num_channel=3):
  nb_samples = 0.
  channel_mean = torch.zeros(num_channel)
  channel_std = torch.zeros(num_channel)
  dataset_len = 0
  dataloader_args = dict(shuffle=False, batch_size=batch_size, num_workers=1, pin_memory=True)
  for i, zip_file in tqdm(enumerate(zip_list)):
    dataset = ZipDataset(zip_file, transform=transforms.Compose([transforms.ToTensor()]))
    dataloader = DataLoader(dataset, **dataloader_args)
    dataset_len += len(dataset)
    for images in dataloader:
      batch_samples = images.size(0)
      # Rearrange batch to be the shape of [B, C, W * H]
      images = images.view(batch_samples, images.size(1), -1)
      channel_mean += images.mean(2).sum(0)
      channel_std += images.std(2).sum(0)
  channel_mean /= dataset_len
  channel_std /= dataset_len

  return {"mean": channel_mean, "std": channel_std, "size": dataset_len}

In [ ]:
import pdb
def run_stastics_for_images(image_list, batch_size=256, num_channel=1):
  dataset_len = 0.
  channel_mean = torch.zeros(num_channel)
  channel_std = torch.zeros(num_channel)
  dataloader_args = dict(shuffle=False, batch_size=100, num_workers=1)
  dataset = CustomDataset(image_list, transform=transforms.Compose([transforms.ToTensor()]))
  dataloader = DataLoader(dataset, **dataloader_args)
  dataset_len += len(dataset)
  for images in dataloader:
      batch_samples = images.size(0)
      # Rearrange batch to be the shape of [B, C, W * H]
      images = images.view(batch_samples, images.size(1), -1)
      channel_mean += images.mean(2).sum(0)
      channel_std += images.std(2).sum(0)
  channel_mean /= dataset_len
  channel_std /= dataset_len

  return {"mean": channel_mean, "std": channel_std, "size": dataset_len}

## **background images mean and std**

In [ ]:
img_list = sorted([f for f in glob.glob('/content/drive/My Drive/Utils/S15A/Images/background/*.jpg')])
val = run_stastics_for_images(img_list, batch_size=50, num_channel=3)
print("bg mean: " , val["mean"].tolist())
print("bg std: " , val["std"].tolist())
print("bg size: " , val["size"])

bg mean:  [0.5558092594146729, 0.5201340913772583, 0.463156521320343]
bg std:  [0.2149990200996399, 0.21596555411815643, 0.23049025237560272]
bg size:  100.0


## **fg_bg images mean and std**

In [ ]:
zip_list = sorted([f for f in glob.glob('/content/gdrive/My Drive/Colab_Notebooks/my_project1/deep_learning/S15_A/images/fg_bg/*.zip')])
val = run_stastics_with_zip_files(zip_list, batch_size=1024, num_channel=3)
print("fg bg mean: " , val["mean"].tolist())
print("fg bg std: " , val["std"].tolist())
print("fg bg size: " , val["size"])

100it [24:54, 14.94s/it]

fg bg mean:  [0.5455222129821777, 0.5086212158203125, 0.45718181133270264]
fg bg std:  [0.22610004246234894, 0.2249932438135147, 0.23590309917926788]
fg bg size:  400000


## **fg_bg_mask images mean and std**

In [ ]:
zip_list = sorted([f for f in glob.glob('/content/gdrive/My Drive/Colab_Notebooks/my_project1/deep_learning/S15_A/images/fg_bg_mask/*.zip')])
val = run_stastics_with_zip_files(zip_list, batch_size=1024, num_channel=1)
print("fg bg mask mean: " , val["mean"].tolist())
print("fg bg mask std: " , val["std"].tolist())
print("fg bg mask size: " , val["size"])

100it [08:44,  5.25s/it]

fg bg mask mean:  [0.05790501832962036]
fg bg mask std:  [0.22068527340888977]
fg bg mask size:  400000


## **dense depth mean and std**

In [ ]:
zip_list = sorted([f for f in glob.glob('/content/gdrive/My Drive/Colab_Notebooks/my_project1/deep_learning/S15_A/images/dense_depth/*.zip')])
val = run_stastics_with_zip_files(zip_list, batch_size=1024, num_channel=1)
print("fg bg dense depth mean: " , val["mean"].tolist())
print("fg bg dense depth std: " , val["std"].tolist())
print("fg bg dense depth size: " , val["size"])

100it [09:26,  5.66s/it]

fg bg dense depth mean:  [0.40361160039901733]
fg bg dense depth std:  [0.19922664761543274]
fg bg dense depth size:  400000
